In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)
# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"
# Common imports
import numpy as np
import os
#imports pandas
import pandas as pd
from IPython.display import Image
import os
#import warnings and Repress Warnings
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
warnings.filterwarnings("ignore", message="numpy.ndarray size changed")
warnings.filterwarnings('ignore')
# to make this notebook's output stable across runs
np.random.seed(42)
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid") #White Grid
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")
import warnings
warnings.filterwarnings('ignore')

1. Download dei dati

In [3]:
#ORIGINAL DATASET https://www.kaggle.com/datasets/usmanshuaibumusa/cicids-17

df1 = pd.read_csv("dataset/archive/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv");
df2 = pd.read_csv("dataset/archive/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv");
df3 = pd.read_csv("dataset/archive/Friday-WorkingHours-Morning.pcap_ISCX.csv");
df4 = pd.read_csv("dataset/archive/Monday-WorkingHours.pcap_ISCX.csv");
df5 = pd.read_csv("dataset/archive/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv");
#df6 = pd.read_csv("dataset/archive/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv");
df7 = pd.read_csv("dataset/archive/Tuesday-WorkingHours.pcap_ISCX.csv");
df8 = pd.read_csv("dataset/archive/Wednesday-workingHours.pcap_ISCX.csv");


In [4]:
df = pd.concat([df1,df2])
del df1,df2
df = pd.concat([df,df3])
del df3
df = pd.concat([df,df4])
del df4
df = pd.concat([df,df5])
del df5
df = pd.concat([df,df7])
del df7
df = pd.concat([df,df8])
del df8

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2660377 entries, 0 to 692702
Data columns (total 85 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   Flow ID                       object 
 1    Source IP                    object 
 2    Source Port                  int64  
 3    Destination IP               object 
 4    Destination Port             int64  
 5    Protocol                     int64  
 6    Timestamp                    object 
 7    Flow Duration                int64  
 8    Total Fwd Packets            int64  
 9    Total Backward Packets       int64  
 10  Total Length of Fwd Packets   float64
 11   Total Length of Bwd Packets  float64
 12   Fwd Packet Length Max        float64
 13   Fwd Packet Length Min        float64
 14   Fwd Packet Length Mean       float64
 15   Fwd Packet Length Std        float64
 16  Bwd Packet Length Max         float64
 17   Bwd Packet Length Min        float64
 18   Bwd Packet Length Mean

In [6]:
pd.DataFrame(df.isnull().sum()/df.shape[0]).sort_values(ascending=0,by=[0])

,0
Flow Bytes/s,0.000503
Flow ID,0.000000
Fwd Avg Bulk Rate,0.000000
Fwd Avg Bytes/Bulk,0.000000
Fwd Header Length.1,0.000000
...,...
Fwd IAT Mean,0.000000
Fwd IAT Total,0.000000
Flow IAT Min,0.000000
Flow IAT Max,0.000000


Elimino righe con Flow Byts/s null

In [8]:
rows = df.loc[df['Flow Bytes/s'].isnull()]
df.drop(labels=rows.index,axis=0,inplace=True)

In [9]:
df.shape

(2652969, 85)

In [10]:
df.describe()

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,2.652969e+06,2.652969e+06,2.652969e+06,2.652969e+06,2.652969e+06,2.652969e+06,2.652969e+06,2.652969e+06,2.652969e+06,2.652969e+06,...,2.652969e+06,2.652969e+06,2.652969e+06,2.652969e+06,2.652969e+06,2.652969e+06,2.652969e+06,2.652969e+06,2.652969e+06,2.652969e+06
mean,4.118867e+04,8.082170e+03,9.794076e+00,1.494647e+07,9.002864e+00,9.919796e+00,5.490700e+02,1.519021e+04,2.102464e+02,1.847228e+01,...,5.008855e+00,-2.927126e+03,8.201064e+04,4.078477e+04,1.524727e+05,5.880923e+04,8.623708e+06,5.277711e+05,9.021780e+06,8.211725e+06
std,2.219300e+04,1.828686e+04,5.234461e+00,3.376538e+07,7.201838e+02,9.581275e+02,1.013298e+04,2.166933e+06,7.305146e+02,6.150985e+01,...,5.979668e+02,1.120752e+06,6.473632e+05,3.948891e+05,1.025976e+06,5.745675e+05,2.413937e+07,4.726771e+06,2.489389e+07,2.387002e+07
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.300000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-5.368707e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.296600e+04,5.300000e+01,6.000000e+00,1.510000e+02,2.000000e+00,1.000000e+00,6.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.094600e+04,8.000000e+01,6.000000e+00,3.132200e+04,2.000000e+00,2.000000e+00,6.200000e+01,1.220000e+02,3.700000e+01,2.000000e+00,...,1.000000e+00,2.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,5.833200e+04,4.430000e+02,1.700000e+01,3.416257e+06,5.000000e+00,4.000000e+00,1.960000e+02,5.100000e+02,8.500000e+01,3.600000e+01,...,2.000000e+00,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.553500e+04,6.553500e+04,1.700000e+01,1.200000e+08,2.197590e+05,2.919220e+05,1.290000e+07,6.554530e+08,2.482000e+04,2.325000e+03,...,2.135570e+05,1.380000e+02,1.070000e+08,7.420000e+07,1.070000e+08,1.070000e+08,1.200000e+08,7.690000e+07,1.200000e+08,1.200000e+08


In [13]:
df[" Label"].value_counts()

BENIGN              2099769
DoS Hulk             229963
PortScan             158233
DDoS                 127612
DoS GoldenEye         10292
FTP-Patator            7934
SSH-Patator            5874
DoS slowloris          5791
DoS Slowhttptest       5495
Bot                    1960
Infiltration             35
Heartbleed               11
Name:  Label, dtype: int64

In [17]:
df_maligne = df[df[" Label"]!='BENIGN']
df_maligne.shape

(553200, 85)

In [18]:
df_maligne[' Destination Port'].value_counts()

80       379523
21         8176
22         6116
8080       1416
444         255
          ...  
53517         1
52348         1
3631          1
51666         1
3051          1
Name:  Destination Port, Length: 1689, dtype: int64